In [ ]:
from matplotlib.dates import date2num, num2date
from matplotlib.colors import ListedColormap
from matplotlib import dates as mdates
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
from matplotlib import ticker

import pandas as pd
import numpy as np
import sys
import os

sys.path.insert(0, "..")
from global_config import config

results_dir   = config.get_property('results_dir')
data_dir      = config.get_property('data_dir')

In [ ]:
usa_df          = pd.read_csv(os.path.join(data_dir, 'processed_data_us.csv'))
dates_forecasts = usa_df.date[11:].values

path_to_frcst_1 = os.path.join(results_dir, "forecast", "arima")
path_to_frcst_2 = os.path.join(results_dir, "forecast", "eakf_model1")
path_to_frcst_3 = os.path.join(results_dir, "forecast", "eakf_model2")
path_to_frcst_4 = os.path.join(results_dir, "forecast", "eakf_model3")
path_to_frcst_5 = os.path.join(results_dir, "forecast", "sarima")



In [ ]:
from utils.utils_eval import compute_evals
from tqdm import tqdm


In [ ]:

usa_df["date"] = pd.to_datetime(usa_df["date"])

evals_df     = []
forecasts_df = []
for idx, date in tqdm(enumerate(dates_forecasts[:-6])):
    ts = pd.to_datetime(str(date))
    d = ts.strftime('%Y-%m-%d')

    frcst1_df = pd.read_csv(os.path.join(path_to_frcst_1, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst2_df = pd.read_csv(os.path.join(path_to_frcst_2, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst3_df = pd.read_csv(os.path.join(path_to_frcst_3, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst4_df = pd.read_csv(os.path.join(path_to_frcst_4, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst5_df = pd.read_csv(os.path.join(path_to_frcst_5, f"{d}.csv"), parse_dates=["date"]).iloc[:6]

    frcst1_df["forecast_date"] = d; frcst1_df["model"] = "arima"
    frcst2_df["forecast_date"] = d; frcst2_df["model"] = "model1"
    frcst3_df["forecast_date"] = d; frcst3_df["model"] = "model2"
    frcst4_df["forecast_date"] = d; frcst4_df["model"] = "model3"
    frcst5_df["forecast_date"] = d; frcst5_df["model"] = "sarima"

    forecasts_df.append(pd.concat([frcst1_df, frcst2_df, frcst3_df, frcst4_df, frcst5_df]))

    dates_use       = frcst2_df.date
    fit_data        = usa_df[usa_df.date.isin((list(dates_use)))].set_index("date")[["total"]]

    eval1_df               = compute_evals(frcst1_df, fit_data)
    eval1_df["frsct_date"] = date
    eval1_df["method"]     = "arima"
    evals_df.append(eval1_df.reset_index())

    eval2_df               = compute_evals(frcst2_df, fit_data)
    eval2_df["frsct_date"] = date
    eval2_df["method"]     = "eakf_model1"
    evals_df.append(eval2_df.reset_index())

    eval3_df               = compute_evals(frcst3_df, fit_data)
    eval3_df["frsct_date"] = date
    eval3_df["method"]     = "eakf_model2"
    evals_df.append(eval3_df.reset_index())

    eval4_df               = compute_evals(frcst4_df, fit_data)
    eval4_df["frsct_date"] = date
    eval4_df["method"]     = "eakf_model3"
    evals_df.append(eval4_df.reset_index())

    eval5_df               = compute_evals(frcst5_df, fit_data)
    eval5_df["frsct_date"] = date
    eval5_df["method"]     = "sarima"
    evals_df.append(eval5_df.reset_index())

forecasts_df = pd.concat(forecasts_df)
evals_df     = pd.concat(evals_df)

forecasts_df.to_csv(os.path.join(results_dir, "forecast", "evaluation", "forecasts.csv"), index=False)
evals_df.to_csv(os.path.join(results_dir, "forecast", "evaluation", "scores.csv"), index=False)


In [ ]:
### eval models with birth and deaths

In [ ]:
usa_df          = pd.read_csv(os.path.join(data_dir, 'processed_data_us.csv'))
dates_forecasts = usa_df.date[11:].values

path_to_frcst_1 = os.path.join(results_dir, "forecast", "arima")
path_to_frcst_2 = os.path.join(results_dir, "forecast", "BD_eakf_model1")
path_to_frcst_3 = os.path.join(results_dir, "forecast", "BD_eakf_model2")
path_to_frcst_4 = os.path.join(results_dir, "forecast", "BD_eakf_model3")
path_to_frcst_5 = os.path.join(results_dir, "forecast", "sarima")



In [ ]:
from utils.utils_eval import compute_evals
from tqdm import tqdm

usa_df["date"] = pd.to_datetime(usa_df["date"])

evals_df     = []
forecasts_df = []
for idx, date in tqdm(enumerate(dates_forecasts[:-6])):
    ts = pd.to_datetime(str(date))
    d = ts.strftime('%Y-%m-%d')

    frcst1_df = pd.read_csv(os.path.join(path_to_frcst_1, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst2_df = pd.read_csv(os.path.join(path_to_frcst_2, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst3_df = pd.read_csv(os.path.join(path_to_frcst_3, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst4_df = pd.read_csv(os.path.join(path_to_frcst_4, f"{d}.csv"), parse_dates=["date"]).iloc[:6]
    frcst5_df = pd.read_csv(os.path.join(path_to_frcst_5, f"{d}.csv"), parse_dates=["date"]).iloc[:6]

    frcst1_df["forecast_date"] = d; frcst1_df["model"] = "arima"
    frcst2_df["forecast_date"] = d; frcst2_df["model"] = "model1"
    frcst3_df["forecast_date"] = d; frcst3_df["model"] = "model2"
    frcst4_df["forecast_date"] = d; frcst4_df["model"] = "model3"
    frcst5_df["forecast_date"] = d; frcst5_df["model"] = "sarima"

    forecasts_df.append(pd.concat([frcst1_df, frcst2_df, frcst3_df, frcst4_df, frcst5_df]))

    dates_use       = frcst2_df.date
    fit_data        = usa_df[usa_df.date.isin((list(dates_use)))].set_index("date")[["total"]]

    eval1_df               = compute_evals(frcst1_df, fit_data)
    eval1_df["frsct_date"] = date
    eval1_df["method"]     = "arima"
    evals_df.append(eval1_df.reset_index())

    eval2_df               = compute_evals(frcst2_df, fit_data)
    eval2_df["frsct_date"] = date
    eval2_df["method"]     = "eakf_model1"
    evals_df.append(eval2_df.reset_index())

    eval3_df               = compute_evals(frcst3_df, fit_data)
    eval3_df["frsct_date"] = date
    eval3_df["method"]     = "eakf_model2"
    evals_df.append(eval3_df.reset_index())

    eval4_df               = compute_evals(frcst4_df, fit_data)
    eval4_df["frsct_date"] = date
    eval4_df["method"]     = "eakf_model3"
    evals_df.append(eval4_df.reset_index())

    eval5_df               = compute_evals(frcst5_df, fit_data)
    eval5_df["frsct_date"] = date
    eval5_df["method"]     = "sarima"
    evals_df.append(eval5_df.reset_index())

forecasts_df = pd.concat(forecasts_df)
evals_df     = pd.concat(evals_df)

forecasts_df.to_csv(os.path.join(results_dir, "forecast", "evaluation", "BD_forecasts.csv"), index=False)
evals_df.to_csv(os.path.join(results_dir, "forecast", "evaluation", "BD_scores.csv"), index=False)
